CNN_Allage_Model for Age detection

In [1]:
import pandas as pd
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.models import Sequential
from keras.layers import InputLayer, Conv2D, MaxPooling2D,GlobalAveragePooling2D,Dropout, Flatten, Dense
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from PIL import Image
import os

In [2]:
path = './dataset/aglined faces/'

In [ ]:
def visualize_image(age, number_of_images):
    plt.figure(figsize=(10, 10))
    folder=path+age+'/'
    images=os.listdir(folder)[:number_of_images]
    for i in range(number_of_images):
        img=mpimg.imread(folder+images[i])
        plt.subplot(number_of_images//2, 2, i+1)
        plt.imshow(img)

visualize_image("020", 10)

In [ ]:
X=[]
y=[]

for folder, _, imgs in os.walk(path):
    if folder!="face_age":
        for img in imgs:
            img_path=folder+'/'+img
            image=Image.open(img_path)
            image=image.convert('L') #gray-scale images
            X.append(np.array(image))
            y.append(int(folder[-3:]))

X=np.array(X)
y=np.array(y)

In [ ]:
X=X.astype("float32")
#normalization
X /= 255.0
y = y//5

In [ ]:
X = X.reshape(X.shape[0], 200, 200, 1)
X.shape

In [ ]:
test_size = 0.3
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, shuffle=True, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [ ]:
train_datagen = ImageDataGenerator(
    shear_range = 0.2, # random application of shearing
    zoom_range = 0.2, 
    horizontal_flip = True) # randomly flipping half of the images horizontally

test_datagen = ImageDataGenerator()

In [ ]:
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from timeit import default_timer as timer

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

early_stopping = EarlyStopping(
                                patience=5, # wait for 5 epochs
                                min_delta = 0.01, # if in 5 epochs the loss function doesn't increase (for accuracy) 
                                               # or decrease (for val_loss) by 1%, then stop
                                verbose=1, # print the training epoch on which training was stopped
                                mode = 'min',
                                monitor='val_loss')

reduce_learning_rate = ReduceLROnPlateau(
                                    monitor="val_loss",
                                    patience=3, # if val_loss plateaus for 3 epochs such that it doesn't see 
                                                # an improvement of size = epsilon
                                    episilon= 0.01,
                                    factor=0.1,  # then we reduce the learning rate by a factor of 0.1
                                    cooldown = 4, # and we wait for 4 epochs before we restart again
                                    verbose=1)

time_callback = TimingCallback()

        
# hyperparameters
lr = 0.01
epochs = 50
batch_size = 128
results = {}
input_shape =[200, 200]

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",
                   padding="valid",
                   kernel_regularizer=regularizers.l2(0.00001),
                   input_shape=(input_shape[0], input_shape[1], 1)))
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(1,activation="linear")) 
    return model

In [ ]:
filepath = 'checkpoints/cnn_face-age__relu.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor="val_loss", save_best_only=True, mode="min", verbose=1)

In [ ]:
model = baseline_model()
model.compile(optimizer="adam", loss="mse", metrics=['mae'])
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='models/cnn_facial_age-relu.png')

In [ ]:
model.fit(
    train_datagen.flow(
        X_train, y_train,
        batch_size = batch_size
    ), # use augmented images
    validation_data = (X_val, y_val),
    steps_per_epoch=X_train.shape[0] // batch_size,
    epochs = epochs,
    callbacks = [
                    reduce_learning_rate,
                    early_stopping,
                    time_callback,
                    checkpoint
                ],
    verbose=True)

In [ ]:
baseline_adam_train_loss = model.history.history["loss"]
baseline_adam_val_loss = model.history.history["val_loss"]
baseline_adam_train_acc = model.history.history["mae"]
baseline_adam_val_acc = model.history.history["val_mae"]

results["baseline_adam"] = {'train-loss': baseline_adam_train_loss,
                             'val-loss': baseline_adam_val_loss,
                             'train-mae': baseline_adam_train_acc,
                             'val-mae': baseline_adam_val_acc}

In [ ]:
plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-mae'])),results[cond]['train-mae'], '-', label=cond+"_train", color="blue")
    plt.plot(range(len(results[cond]['val-mae'])),results[cond]['val-mae'], '-', label=cond+"_val", color="green")
plt.title("Mean Absolute Error")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
model_test_results = model.evaluate(X_test, y_test, batch_size=128)
dict(zip(model.metrics_names, model_test_results))

In [ ]:
y_pred = model.predict(X).reshape(-1)

In [ ]:
y_pred_range=np.round((y_pred), 0).astype(int)
float(100*sum(np.equal(y, y_pred_range).astype(int))/len(y_pred_range))

In [ ]:
pd.Series((5 * y_pred_range.reshape((len(y_pred))))).plot(kind='hist', bins=40, label='Predicted', alpha=0.5)
pd.Series((5 * y)).plot(kind='hist', bins=40, label='Original', alpha=0.5)

plt.legend(title='Group')

plt.ylabel('Frequency', fontsize=14)
plt.xlabel('Age', fontsize=14)
plt.title('Predicted vs Original', fontsize=16)

In [ ]:
import seaborn as sns
sns.heatmap(confusion_matrix(y, y_pred_range))

In [ ]:
print(classification_report(y, y_pred_range))